<a href="https://colab.research.google.com/github/fininsight/text-mining-tutorial/blob/master/3_%ED%95%B5%EC%8B%AC_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C_Keyword_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 핵심 키워드 추출 (Keyword Extraction)

# 1 TF-IDF

## 1.1 샘플 텍스트

In [0]:
d1 = "The cat sat on my face. I hate a cat."
d2 = "The dog sat on my bed. I love a dog." 

## 1.2 sklearn 활용 TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

document_ls = [d1, d2]

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(document_ls)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

## 1.3 dataframe으로 변환하여 출력

In [99]:
import pandas as pd
count_vect_df = pd.DataFrame(tfidf.todense(), columns=vectorizer.get_feature_names())
count_vect_df

,bed,cat,dog,face,hate,love,my,on,sat,the
0,0.000000,0.706006,0.000000,0.353003,0.353003,0.000000,0.251164,0.251164,0.251164,0.251164
1,0.353003,0.000000,0.706006,0.000000,0.000000,0.353003,0.251164,0.251164,0.251164,0.251164


## 1.4 TF-IDF score가 높은 순으로 출력

In [111]:
feature_array = np.array(vectorizer.get_feature_names())
tfidf_sorting = np.argsort(tfidf[0].toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]

top_n

array(['cat', 'hate', 'face'], dtype='<U4')



---



# 2 Textrank

<img src="https://i.stack.imgur.com/ohF5r.png" />

## 2.3 gensim Textrank

In [0]:
from gensim.summarization import keywords

text = '''Challenges in natural language processing frequently involve speech recognition, natural language understanding, natural language generation (frequently from formal, machine-readable logical forms), connecting language and machine perception, dialog systems, or some combination thereof.'''
keywords(text).split('\n')

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


['natural language', 'machine', 'frequently']

In [0]:
from gensim.summarization.summarizer import summarize

text = '''Rice Pudding - Poem by Alan Alexander Milne
... What is the matter with Mary Jane?
... She's crying with all her might and main,
... And she won't eat her dinner - rice pudding again -
... What is the matter with Mary Jane?
... What is the matter with Mary Jane?
... I've promised her dolls and a daisy-chain,
... And a book about animals - all in vain -
... What is the matter with Mary Jane?
... What is the matter with Mary Jane?
... She's perfectly well, and she hasn't a pain;
... But, look at her, now she's beginning again! -
... What is the matter with Mary Jane?
... What is the matter with Mary Jane?
... I've promised her sweets and a ride in the train,
... And I've begged her to stop for a bit and explain -
... What is the matter with Mary Jane?
... What is the matter with Mary Jane?
... She's perfectly well and she hasn't a pain,
... And it's lovely rice pudding for dinner again!
... What is the matter with Mary Jane?'''
print(summarize(text))


And she won't eat her dinner - rice pudding again -
I've promised her dolls and a daisy-chain,
I've promised her sweets and a ride in the train,
And it's lovely rice pudding for dinner again!


## 2.1 TextRank 직접 구현하기
(Based on: https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

In [0]:
#Source of text:
#https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents

Text = "Compatibility of systems of linear constraints over the set of natural numbers. \
Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and \
nonstrict inequations are considered. \
Upper bounds for components of a minimal set of solutions and \
algorithms of construction of minimal generating sets of solutions for all \
types of systems are given. \
These criteria and the corresponding algorithms for constructing \
a minimal supporting set of solutions can be used in solving all the \
considered types of systems and systems of mixed types."

Text = "북한이 4일 오전 강원도 원산 호도반도 일대에서 여러 발의 단거리 미사일을 발사한 가운데 탄도미사일이 아닌 방사포(다연장로켓)나 전술 로켓일 가능성이 제기된다. \
합동참모본부는 이날 '북한이 오전 9시6분께부터 9시27분께까지 원산북방 호도반도 일대에서 북동쪽 방향으로 불상 단거리 발사체 수발을 발사했다'고 밝혔다. \
발사된 발사체는 동해상까지 약 70~200㎞ 비행했으며 미사일 발사 의도와 기종 등 추가정보에 대해서는 한미가 정밀분석 중에 있다. \
당초 합참은 북한이 '단거리 미사일'을 발사했다고 발표했다가 40여분 만에 '단거리 발사체'로 수정 발표했다. 미사일로 보기 어렵다는 분석을 담은 것으로 보여진다. 유엔 제재는 모든 탄도미사일 발사를 금지하고 있다. \
북한은 지난달 17일에도 '신형 전술유도무기'라는 표현으로 단거리 발사체를 발사한 바 있다. 당시 군은 이에 대해 '지상전투용 유도무기'로 분석했다. 먼거리를 날아가는 순항미사일이 아닌 사거리 20여㎞ 수준의 단거리 발사체로 본 셈이다. \
북한은 정확히 17일 만에 단거리 발사체를 발사했는데 전문가들에 따르면 이번 발사체들은 240㎜ 방사포와 300㎜ 방사포일 것으로 추정된다. \
발사 수와 비행거리를 고려했을 때 240㎜방사포와 300㎜신형방사포를 함께 가지고 나와 최대사거리를 뽐내는 실발사훈련을 했을 가능성이 높아보인다는 분석이다. \
김동엽 경남대 극동문제연구소 교수는 '일단 장사정포, 일명 다연장로켓, 북한에선 방사포라고 하는 것의  실제훈련을 한 것이 아닐까 한다'고 말했다. \
이어 \방사포를 탄도미사일로 볼 것인가는 해석의 차이가 있겠지만 일단 발사 방식상 탄도지만 미사일로까지 보기는 어렵다는 점에서 이번 북한의 행동을 모든 탄도미사일 발사를 금지하고 있는 유엔제재 위반으로 보기는 어려울 듯 하다'고 부연했다. \
홍민 통일연구원 북한연구실장은 'KN-09와 같은 대구경 방사포 또는 신형 함대함 미사일일 수 있다'고 말했다. 2015년 6월14일에도 북한은 신형 함대함 미사일 발사 훈련을 호도반도에서 진행한 바 있다. \
다만 일각에선 사거리가 200㎞까지 나온 것에 비춰볼 때 단거리 미사일의 가능성을 배제할 수 없다는 주장도 나온다. \
양욱 한국국방안보포럼 센터장은 '애초에 300㎜ 방사포란 것은 탄도미사일로 보아도 무방한, 즉 SRBM(단거리 탄도미사일)과 MLRS(다연장 로켓)의 경계선 상에 존재하는 무기체계'라며 '단거리 미사일의 가능성을 배제할 수 없다'고 밝혔다. \
2018년 2월 열병식에 등장했던 KN-02 단거리미사일의 개량형일 가능성을 추정할 수 있으며, '방사포'라고 하더라도 KN-09이거나 또는 그 개량형일 가능성도 있다는 주장이다."

### Cleaning Text Data

The raw input text is cleaned off non-printable characters (if any) and turned into lower case.
The processed input text is then tokenized using NLTK library functions. 

In [57]:
import nltk
from nltk import word_tokenize
import string

nltk.download('punkt')


#def clean(text):
#    text = text.lower()
#    printable = set(string.printable)    
#    text = filter(lambda x: (x in printable), text)
#    print(text)
#    return text

#Cleaned_text = clean(Text)
#print(Cleaned_text)

text = word_tokenize(Text)

print ("Tokenized Text: \n")
print (text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Tokenized Text: 

['북한이', '4일', '오전', '강원도', '원산', '호도반도', '일대에서', '여러', '발의', '단거리', '미사일을', '발사한', '가운데', '탄도미사일이', '아닌', '방사포', '(', '다연장로켓', ')', '나', '전술', '로켓일', '가능성이', '제기된다', '.', '합동참모본부는', '이날', "'북한이", '오전', '9시6분께부터', '9시27분께까지', '원산북방', '호도반도', '일대에서', '북동쪽', '방향으로', '불상', '단거리', '발사체', '수발을', "발사했다'고", '밝혔다', '.', '발사된', '발사체는', '동해상까지', '약', '70~200㎞', '비행했으며', '미사일', '발사', '의도와', '기종', '등', '추가정보에', '대해서는', '한미가', '정밀분석', '중에', '있다', '.', '당초', '합참은', '북한이', "'단거리", "미사일'을", '발사했다고', '발표했다가', '40여분', '만에', "'단거리", "발사체'로", '수정', '발표했다', '.', '미사일로', '보기', '어렵다는', '분석을', '담은', '것으로', '보여진다', '.', '유엔', '제재는', '모든', '탄도미사일', '발사를', '금지하고', '있다', '.', '북한은', '지난달', '17일에도', "'신형", "전술유도무기'라는", '표현으로', '단거리', '발사체를', '발사한', '바', '있다', '.', '당시', '군은', '이에', '대해', "'지상전투용", "유도무기'로", '분석했다', '.', '먼거리를', '날아가는', '순항미사일이', '아닌', '사거리', '20여㎞', '수준의', '단거리', '발사체로', 

### POS Tagging For Lemmatization

NLTK is again used for <b>POS tagging</b> the input text so that the words can be lemmatized based on their POS tags.

Description of POS tags: 


http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [58]:
nltk.download('averaged_perceptron_tagger')
  
POS_tag = nltk.pos_tag(text)

print("Tokenized Text with POS tags: \n")
print(POS_tag)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Tokenized Text with POS tags: 

[('북한이', 'RB'), ('4일', 'CD'), ('오전', 'JJ'), ('강원도', 'NNP'), ('원산', 'NNP'), ('호도반도', 'NNP'), ('일대에서', 'NNP'), ('여러', 'NNP'), ('발의', 'NNP'), ('단거리', 'NNP'), ('미사일을', 'NNP'), ('발사한', 'NNP'), ('가운데', 'NNP'), ('탄도미사일이', 'NNP'), ('아닌', 'NNP'), ('방사포', 'NNP'), ('(', '('), ('다연장로켓', 'NNP'), (')', ')'), ('나', 'VBP'), ('전술', 'JJ'), ('로켓일', 'NNP'), ('가능성이', 'NNP'), ('제기된다', 'NNP'), ('.', '.'), ('합동참모본부는', 'VB'), ('이날', 'JJ'), ("'북한이", 'NNP'), ('오전', 'NNP'), ('9시6분께부터', 'CD'), ('9시27분께까지', 'CD'), ('원산북방', 'NN'), ('호도반도', 'NNP'), ('일대에서', 'NNP'), ('북동쪽', 'NNP'), ('방향으로', 'NNP'), ('불상', 'NNP'), ('단거리', 'NNP'), ('발사체', 'NNP'), ('수발을', 'NNP'), ("발사했다'고", 'NNP'), ('밝혔다', 'NNP'), ('.', '.'), ('발사된', 'VB'), ('발사체는', 'JJ'), ('동해상까지', 'NNP'), ('약', 'NNP'), ('70~200㎞', 'CD'), ('비행했으며', 'NNP'

### Lemmatization

The tokenized text (mainly the nouns and adjectives) is normalized by <b>lemmatization</b>.
In lemmatization different grammatical counterparts of a word will be replaced by single
basic lemma. For example, 'glasses' may be replaced by 'glass'. 

Details about lemmatization: 
    
https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

In [59]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
print("Text tokens after lemmatization of adjectives and nouns: \n")
print(lemmatized_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Text tokens after lemmatization of adjectives and nouns: 

['북한이', '4일', '오전', '강원도', '원산', '호도반도', '일대에서', '여러', '발의', '단거리', '미사일을', '발사한', '가운데', '탄도미사일이', '아닌', '방사포', '(', '다연장로켓', ')', '나', '전술', '로켓일', '가능성이', '제기된다', '.', '합동참모본부는', '이날', "'북한이", '오전', '9시6분께부터', '9시27분께까지', '원산북방', '호도반도', '일대에서', '북동쪽', '방향으로', '불상', '단거리', '발사체', '수발을', "발사했다'고", '밝혔다', '.', '발사된', '발사체는', '동해상까지', '약', '70~200㎞', '비행했으며', '미사일', '발사', '의도와', '기종', '등', '추가정보에', '대해서는', '한미가', '정밀분석', '중에', '있다', '.', '당초', '합참은', '북한이', "'단거리", "미사일'을", '발사했다고', '발표했다가', '40여분', '만에', "'단거리", "발사체'로", '수정', '발표했다', '.', '미사일로', '보기', '어렵다는', '분석을', '담은', '것으로', '보여진다', '.', '유엔', '제재는', '모든', '탄도미사일', '발사를', '금지하고', '있다', '.', '북한은', '지난달', '17일에도', "'신형", "전술유도무기'라는", '표현으로', '단거리', '발사체를', '발사한', '바', '있다', '.', '당시', '군은', '이에', '대해', "'지상전투용", "유도무기'로", '분석했다', '.', '먼거리를', '날아가는', '순항미사일이'

### POS tagging for Filtering

The <b>lemmatized text</b> is <b>POS tagged</b> here. The tags will be used for filtering later on.

In [60]:
POS_tag = nltk.pos_tag(lemmatized_text)

print("Lemmatized text with POS tags: \n")
print(POS_tag)

Lemmatized text with POS tags: 

[('북한이', 'RB'), ('4일', 'CD'), ('오전', 'JJ'), ('강원도', 'NNP'), ('원산', 'NNP'), ('호도반도', 'NNP'), ('일대에서', 'NNP'), ('여러', 'NNP'), ('발의', 'NNP'), ('단거리', 'NNP'), ('미사일을', 'NNP'), ('발사한', 'NNP'), ('가운데', 'NNP'), ('탄도미사일이', 'NNP'), ('아닌', 'NNP'), ('방사포', 'NNP'), ('(', '('), ('다연장로켓', 'NNP'), (')', ')'), ('나', 'VBP'), ('전술', 'JJ'), ('로켓일', 'NNP'), ('가능성이', 'NNP'), ('제기된다', 'NNP'), ('.', '.'), ('합동참모본부는', 'VB'), ('이날', 'JJ'), ("'북한이", 'NNP'), ('오전', 'NNP'), ('9시6분께부터', 'CD'), ('9시27분께까지', 'CD'), ('원산북방', 'NN'), ('호도반도', 'NNP'), ('일대에서', 'NNP'), ('북동쪽', 'NNP'), ('방향으로', 'NNP'), ('불상', 'NNP'), ('단거리', 'NNP'), ('발사체', 'NNP'), ('수발을', 'NNP'), ("발사했다'고", 'NNP'), ('밝혔다', 'NNP'), ('.', '.'), ('발사된', 'VB'), ('발사체는', 'JJ'), ('동해상까지', 'NNP'), ('약', 'NNP'), ('70~200㎞', 'CD'), ('비행했으며', 'NNP'), ('미사일', 'NNP'), ('발사', 'NNP'), ('의도와', 'NNP'), ('기종', 'NNP'), ('등', 'NNP'), ('추가정보에', 'NNP'), ('대해서는', 'NNP'), ('한미가', 'NNP'), ('정밀분석', 'NNP'), ('중에', 'NNP'), ('있다', 'NNP'), ('.', '.')

## POS Based Filtering

Any word from the lemmatized text, which isn't a noun, adjective, or gerund (or a 'foreign word'), is here
considered as a <b>stopword</b> (non-content). This is based on the assumption that usually keywords are noun,
adjectives or gerunds. 

Punctuations are added to the stopword list too.

In [0]:
stopwords = []

wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

### Complete stopword generation

Even if we remove the aforementioned stopwords, still some extremely common nouns, adjectives or gerunds may
remain which are very bad candidates for being keywords (or part of it). 

An external file constituting a long list of stopwords is loaded and all the words are added with the previous
stopwords to create the final list 'stopwords-plus' which is then converted into a set. 

(Source of stopwords data: https://www.ranks.nl/stopwords)

Stopwords-plus constitute the sum total of all stopwords and potential phrase-delimiters. 

(The contents of this set will be later used to partition the lemmatized text into n-gram phrases. But, for now, I will simply remove the stopwords, and work with a 'bag-of-words' approach. I will be developing the graph using unigram texts as vertices)

In [0]:
#stopword_file = open("long_stopwords.txt", "r")
#Source = https://www.ranks.nl/stopwords

lots_of_stopwords = []

#for line in stopword_file.readlines():
#    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

#Stopwords_plus contain total set of all stopwords

### Removing Stopwords 

Removing stopwords from lemmatized_text. 
Processeced_text condtains the result.

In [63]:
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
print(processed_text)

['오전', '강원도', '원산', '호도반도', '일대에서', '여러', '발의', '단거리', '미사일을', '발사한', '가운데', '탄도미사일이', '아닌', '방사포', '다연장로켓', '전술', '로켓일', '가능성이', '제기된다', '이날', "'북한이", '오전', '원산북방', '호도반도', '일대에서', '북동쪽', '방향으로', '불상', '단거리', '발사체', '수발을', "발사했다'고", '밝혔다', '발사체는', '동해상까지', '약', '비행했으며', '미사일', '의도와', '기종', '등', '추가정보에', '대해서는', '한미가', '정밀분석', '중에', '있다', '합참은', "미사일'을", '발사했다고', '만에', "발사체'로", '수정', '발표했다', '보기', '어렵다는', '분석을', '담은', '것으로', '보여진다', '제재는', '모든', '탄도미사일', '발사를', '금지하고', '있다', '지난달', "전술유도무기'라는", '표현으로', '단거리', '발사체를', '발사한', '바', '있다', '군은', '이에', '대해', "유도무기'로", '분석했다', '날아가는', '순항미사일이', '아닌', '수준의', '단거리', '발사체로', '본', '셈이다', '정확히', '만에', '단거리', '발사체를', '발사했는데', '전문가들에', '따르면', '이번', '방사포와', '방사포일', '것으로', '추정된다', '수와', '비행거리를', '고려했을', '함께', '가지고', '나와', '최대사거리를', '뽐내는', '실발사훈련을', '했을', '가능성이', '높아보인다는', '분석이다', '경남대', '극동문제연구소', '교수는', '장사정포', '일명', '다연장로켓', '북한에선', '방사포라고', '하는', '것의', '실제훈련을', '한', '것이', '아닐까', "한다'고", '말했다', '\\방사포를', '탄도미사일로', '볼', '것인가는', '해석의', '차이가', '있겠지만', 

## Vocabulary Creation

Vocabulary will only contain unique words from processed_text.

In [64]:
vocabulary = list(set(processed_text))
print(vocabulary)

['미사일의', '듯', '일대에서', '지난달', '대해', '강원도', '표현으로', '있다', '방향으로', '탄도미사일', '것으로', '가지고', '신형', '함대함', '발사체는', '발사했는데', '진행한', '했을', '제기된다', '발사체', '방사포라고', "하다'고", '원산', '발의', '일각에선', '수', '발표했다', '부연했다', '나온', '미사일일', '위반으로', '보기는', "한다'고", '말했다', '하더라도', '방식상', '가운데', '나온다', "'북한이", '높아보인다는', '것인가는', "미사일'을", '분석을', '가능성을', '분석했다', '여러', '발사했다고', '한', '등장했던', '실발사훈련을', '북동쪽', '대해서는', '유엔제재', '것에', '미사일을', "발사체'로", '일명', '이에', "유도무기'로", '대구경', '수정', '다연장', '의도와', '함께', '추정된다', '통일연구원', '북한에선', '실제훈련을', '장사정포', '일단', '다연장로켓', '원산북방', '미사일', '있는', '그', '방사포란', '셈이다', '\\방사포를', '탄도미사일로', '한국국방안보포럼', '단거리미사일의', '같은', '보기', '수발을', '존재하는', "있다'고", '차이가', '가능성도', '볼', '탄도지만', '무방한', '로켓일', '보여진다', '호도반도에서', '고려했을', '있겠지만', '상에', "없다'고", 'KN-09이거나', "발사했다'고", '것의', '순항미사일이', '바', '아닐까', '동해상까지', '만에', '따르면', '발사한', '방사포와', '오전', 'KN-02', '단거리', '비행했으며', '수준의', '북한연구실장은', '전술', '열병식에', '밝혔다', '발사체로', '즉', '경남대', '주장이다', '날아가는', '어렵다는', '호도반도', '북한의', '약', '극동문제연구소', '탄도미사일이', '있으며', '미사일로까지', '불

### Building Graph

TextRank is a graph based model, and thus it requires us to build a graph. Each words in the vocabulary will serve as a vertex for graph. The words will be represented in the vertices by their index in vocabulary list.  

The weighted_edge matrix contains the information of edge connections among all vertices.
I am building wieghted undirected edges.

weighted_edge[i][j] contains the weight of the connecting edge between the word vertex represented by vocabulary index i and the word vertex represented by vocabulary j.

If weighted_edge[i][j] is zero, it means no edge connection is present between the words represented by index i and j.

There is a connection between the words (and thus between i and j which represents them) if the words co-occur within a window of a specified 'window_size' in the processed_text.

The value of the weighted_edge[i][j] is increased by (1/(distance between positions of words currently represented by i and j)) for every connection discovered between the same words in different locations of the text. 

The covered_coocurrences list (which is contain the list of pairs of absolute positions in processed_text of the words whose coocurrence at that location is already checked) is managed so that the same two words located in the same positions in processed_text are not repetitively counted while sliding the window one text unit at a time.

The score of all vertices are intialized to one. 

Self-connections are not considered, so weighted_edge[i][i] will be zero.

In [0]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])


### Calculating weighted summation of connections of a vertex

inout[i] will contain the sum of all the undirected connections\edges associated withe the vertex represented by i.

In [0]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

### Scoring Vertices

The formula used for scoring a vertex represented by i is:

score[i] = (1-d) + d x [ Summation(j) ( (weighted_edge[i][j]/inout[j]) x score[j] ) ] where j belongs to the list of vertieces that has a connection with i. 

d is the damping factor.

The score is iteratively updated until convergence. 

In [67]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print("Converging at iteration "+str(iter)+"....")
        break


Converging at iteration 28....


In [68]:
for i in range(0,vocab_len):
    print("Score of "+vocabulary[i]+": "+str(score[i]))

Score of 미사일의: 1.3394127
Score of 듯: 0.9122603
Score of 일대에서: 1.4795749
Score of 지난달: 0.7699191
Score of 대해: 0.8938158
Score of 강원도: 0.7221002
Score of 표현으로: 0.76401246
Score of 있다: 2.7670085
Score of 방향으로: 0.8012033
Score of 탄도미사일: 1.9642564
Score of 것으로: 1.5896914
Score of 가지고: 0.9738205
Score of 신형: 1.379112
Score of 함대함: 1.3798486
Score of 발사체는: 0.854603
Score of 발사했는데: 0.78580105
Score of 진행한: 0.78955317
Score of 했을: 0.91731805
Score of 제기된다: 0.8826324
Score of 발사체: 0.79881024
Score of 방사포라고: 0.91809136
Score of 하다'고: 0.9323855
Score of 원산: 0.8283305
Score of 발의: 0.7694295
Score of 일각에선: 0.7883215
Score of 수: 2.6255715
Score of 발표했다: 0.8698311
Score of 부연했다: 0.9381928
Score of 나온: 0.8091199
Score of 미사일일: 0.7348361
Score of 위반으로: 0.84753454
Score of 보기는: 1.6039605
Score of 한다'고: 0.8618041
Score of 말했다: 1.4703345
Score of 하더라도: 0.79689914
Score of 방식상: 0.9384601
Score of 가운데: 0.7782201
Score of 나온다: 0.8914458
Score of '북한이: 0.87239516
Score of 높아보인다는: 0.91330594
Score of 것인가는: 0.91

### Phrase Partiotioning

Paritioning lemmatized_text into phrases using the stopwords in it as delimeters.
The phrases are also candidates for keyphrases to be extracted. 

In [70]:
phrases = []

phrase = " "
for word in lemmatized_text:
    
    if word in stopwords_plus:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords_plus:
        phrase+=str(word)
        phrase+=" "

print("Partitioned Phrases (Candidate Keyphrases): \n")
print(phrases)

Partitioned Phrases (Candidate Keyphrases): 

[['오전', '강원도', '원산', '호도반도', '일대에서', '여러', '발의', '단거리', '미사일을', '발사한', '가운데', '탄도미사일이', '아닌', '방사포'], ['다연장로켓'], ['전술', '로켓일', '가능성이', '제기된다'], ['이날', "'북한이", '오전'], ['원산북방', '호도반도', '일대에서', '북동쪽', '방향으로', '불상', '단거리', '발사체', '수발을', "발사했다'고", '밝혔다'], ['발사체는', '동해상까지', '약'], ['비행했으며', '미사일'], ['의도와', '기종', '등', '추가정보에', '대해서는', '한미가', '정밀분석', '중에', '있다'], ['합참은'], ["미사일'을", '발사했다고'], ['만에'], ["발사체'로", '수정', '발표했다'], ['보기', '어렵다는', '분석을', '담은', '것으로', '보여진다'], ['제재는', '모든', '탄도미사일', '발사를', '금지하고', '있다'], ['지난달'], ["전술유도무기'라는", '표현으로', '단거리', '발사체를', '발사한', '바', '있다'], ['군은', '이에', '대해'], ["유도무기'로", '분석했다'], ['날아가는', '순항미사일이', '아닌'], ['수준의', '단거리', '발사체로', '본', '셈이다'], ['정확히'], ['만에', '단거리', '발사체를', '발사했는데', '전문가들에', '따르면', '이번'], ['방사포와'], ['방사포일', '것으로', '추정된다'], ['수와', '비행거리를', '고려했을'], ['함께', '가지고', '나와', '최대사거리를', '뽐내는', '실발사훈련을', '했을', '가능성이', '높아보인다는', '분석이다'], ['경남대', '극동문제연구소', '교수는'], ['장사정포'], ['일명', '다연장로켓'], ['북한에선', '방사포라고', '하는'

### Create a list of unique phrases.

Repeating phrases\keyphrase candidates has no purpose here, anymore. 

In [71]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print("Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Unique Phrases (Candidate Keyphrases): 

[['오전', '강원도', '원산', '호도반도', '일대에서', '여러', '발의', '단거리', '미사일을', '발사한', '가운데', '탄도미사일이', '아닌', '방사포'], ['다연장로켓'], ['전술', '로켓일', '가능성이', '제기된다'], ['이날', "'북한이", '오전'], ['원산북방', '호도반도', '일대에서', '북동쪽', '방향으로', '불상', '단거리', '발사체', '수발을', "발사했다'고", '밝혔다'], ['발사체는', '동해상까지', '약'], ['비행했으며', '미사일'], ['의도와', '기종', '등', '추가정보에', '대해서는', '한미가', '정밀분석', '중에', '있다'], ['합참은'], ["미사일'을", '발사했다고'], ['만에'], ["발사체'로", '수정', '발표했다'], ['보기', '어렵다는', '분석을', '담은', '것으로', '보여진다'], ['제재는', '모든', '탄도미사일', '발사를', '금지하고', '있다'], ['지난달'], ["전술유도무기'라는", '표현으로', '단거리', '발사체를', '발사한', '바', '있다'], ['군은', '이에', '대해'], ["유도무기'로", '분석했다'], ['날아가는', '순항미사일이', '아닌'], ['수준의', '단거리', '발사체로', '본', '셈이다'], ['정확히'], ['만에', '단거리', '발사체를', '발사했는데', '전문가들에', '따르면', '이번'], ['방사포와'], ['방사포일', '것으로', '추정된다'], ['수와', '비행거리를', '고려했을'], ['함께', '가지고', '나와', '최대사거리를', '뽐내는', '실발사훈련을', '했을', '가능성이', '높아보인다는', '분석이다'], ['경남대', '극동문제연구소', '교수는'], ['장사정포'], ['일명', '다연장로켓'], ['북한에선', '방사포라고', '하는', '것의

### Thinning the list of candidate-keyphrases.

Removing single word keyphrases-candidates that are present multi-word alternatives. 

In [72]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print("Thinned Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Thinned Unique Phrases (Candidate Keyphrases): 

[['오전', '강원도', '원산', '호도반도', '일대에서', '여러', '발의', '단거리', '미사일을', '발사한', '가운데', '탄도미사일이', '아닌', '방사포'], ['전술', '로켓일', '가능성이', '제기된다'], ['이날', "'북한이", '오전'], ['원산북방', '호도반도', '일대에서', '북동쪽', '방향으로', '불상', '단거리', '발사체', '수발을', "발사했다'고", '밝혔다'], ['발사체는', '동해상까지', '약'], ['비행했으며', '미사일'], ['의도와', '기종', '등', '추가정보에', '대해서는', '한미가', '정밀분석', '중에', '있다'], ['합참은'], ["미사일'을", '발사했다고'], ["발사체'로", '수정', '발표했다'], ['보기', '어렵다는', '분석을', '담은', '것으로', '보여진다'], ['제재는', '모든', '탄도미사일', '발사를', '금지하고', '있다'], ['지난달'], ["전술유도무기'라는", '표현으로', '단거리', '발사체를', '발사한', '바', '있다'], ['군은', '이에', '대해'], ["유도무기'로", '분석했다'], ['날아가는', '순항미사일이', '아닌'], ['수준의', '단거리', '발사체로', '본', '셈이다'], ['정확히'], ['만에', '단거리', '발사체를', '발사했는데', '전문가들에', '따르면', '이번'], ['방사포와'], ['방사포일', '것으로', '추정된다'], ['수와', '비행거리를', '고려했을'], ['함께', '가지고', '나와', '최대사거리를', '뽐내는', '실발사훈련을', '했을', '가능성이', '높아보인다는', '분석이다'], ['경남대', '극동문제연구소', '교수는'], ['장사정포'], ['일명', '다연장로켓'], ['북한에선', '방사포라고', '하는', '것의', '실제훈련을',

### Scoring Keyphrases

Scoring the phrases (candidate keyphrases) and building up a list of keyphrases\keywords
by listing untokenized versions of tokenized phrases\candidate-keyphrases.
Phrases are scored by adding the score of their members (words\text-units that were ranked by the graph algorithm)


In [73]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    i+=1

Keyword: '오전 강원도 원산 호도반도 일대에서 여러 발의 단거리 미사일을 발사한 가운데 탄도미사일이 아닌 방사포', Score: 18.59939581155777
Keyword: '전술 로켓일 가능성이 제기된다', Score: 4.263720273971558
Keyword: '이날 '북한이 오전', Score: 2.977706551551819
Keyword: '원산북방 호도반도 일대에서 북동쪽 방향으로 불상 단거리 발사체 수발을 발사했다'고 밝혔다', Score: 14.819874942302704
Keyword: '발사체는 동해상까지 약', Score: 2.5923511385917664
Keyword: '비행했으며 미사일', Score: 2.387024223804474
Keyword: '의도와 기종 등 추가정보에 대해서는 한미가 정밀분석 중에 있다', Score: 9.945255756378174
Keyword: '합참은', Score: 0.8144996166229248
Keyword: '미사일'을 발사했다고', Score: 1.649080514907837
Keyword: '발사체'로 수정 발표했다', Score: 2.581561803817749
Keyword: '보기 어렵다는 분석을 담은 것으로 보여진다', Score: 6.516274154186249
Keyword: '제재는 모든 탄도미사일 발사를 금지하고 있다', Score: 9.545214354991913
Keyword: '지난달', Score: 0.7699190974235535
Keyword: '전술유도무기'라는 표현으로 단거리 발사체를 발사한 바 있다', Score: 13.090409815311432
Keyword: '군은 이에 대해', Score: 2.5699865221977234
Keyword: '유도무기'로 분석했다', Score: 1.808154284954071
Keyword: '날아가는 순항미사일이 아닌', Score: 3.2142947912216187
Keyword: '수준의 단거리 발

### Ranking Keyphrases

Ranking keyphrases based on their calculated scores. Displaying top keywords_num no. of keyphrases.

In [74]:
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = 10

print("Keywords:\n")

for i in range(0,keywords_num):
    print(str(keywords[sorted_index[i]])+", ")

Keywords:

방식상 탄도지만 미사일로까지 보기는 어렵다는 점에서 이번 북한의 행동을 모든 탄도미사일 발사를 금지하고 있는 유엔제재 위반으로 보기는 어려울 듯 하다'고 부연했다, 
오전 강원도 원산 호도반도 일대에서 여러 발의 단거리 미사일을 발사한 가운데 탄도미사일이 아닌 방사포, 
원산북방 호도반도 일대에서 북동쪽 방향으로 불상 단거리 발사체 수발을 발사했다'고 밝혔다, 
단거리 미사일의 가능성을 배제할 수 없다는 주장도 나온다, 
전술유도무기'라는 표현으로 단거리 발사체를 발사한 바 있다, 
같은 대구경 방사포 또는 신형 함대함 미사일일 수 있다'고 말했다, 
만에 단거리 발사체를 발사했는데 전문가들에 따르면 이번, 
열병식에 등장했던 KN-02 단거리미사일의 개량형일 가능성을 추정할 수 있으며, 
함께 가지고 나와 최대사거리를 뽐내는 실발사훈련을 했을 가능성이 높아보인다는 분석이다, 
의도와 기종 등 추가정보에 대해서는 한미가 정밀분석 중에 있다, 


# Input:

Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types of systems and systems of mixed types.

# Extracted Keywords:

* minimal supporting set,  
* minimal generating set,  
* minimal set,  
* linear diophantine equation,  
* nonstrict inequations,  
* strict inequations,  
* system,  
* linear constraint,  
* solution,  
* upper bound, 
